In [44]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --cluster-id j-30GQL37TFA5B5 --auth-type None  

The sagemaker_studio_analytics_extension.magics extension is already loaded. To reload it, use:
  %reload_ext sagemaker_studio_analytics_extension.magics
Initiating EMR connection..
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1649316171984_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
{"namespace": "sagemaker-analytics", "cluster_id": "j-30GQL37TFA5B5", "error_message": null, "success": true, "service": "emr", "operation": "connect"}


#### 注意事项

您需要访问设置为使用 Amazon Virtual Private Cloud (Amazon VPC) 模式的 SageMaker Studio。

SageMaker Studio 使用的所有子网必须是私有子网。

如果您将sm-analytics实用程序来配置 SparkMagic 内核，请遵循以下两个先决条件之一：

- 确保 Amazon VPC 接口终端节点已连接到 SageMaker Studio 使用的所有子网。

- 确保 SageMaker Studio 使用的所有子网都路由为使用 NAT 网关。有关更多信息，请参阅 。NAT 网关.

如果以下任一点适用于您，则在使用亚马逊 EMR 时必须安装 Spark 和 Livy。

- 您的亚马逊 EMR 集群与 Studio 位于同一亚马逊 VPC 中。

- 您的集群位于连接到 Studio 中的 Amazon VPC 中的亚马逊 VPC 中。

Amazon SageMaker Studio 和亚马逊 EMR 的安全组必须允许相互访问和相互访问。

您的亚马逊 EMR 安全组必须打开端口 8998，以便 Amazon SageMaker Studio 可以通过 Livy 与 Spark 集群进行通信。

In [45]:
from pyspark.sql.session import SparkSession

spark = SparkSession(sc)
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 处理原始数据 

将S3导出数据进行处理


In [51]:
df=spark.read.csv("s3://my-ml-buckets/workshop-data/", header=True)
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+--------------------+----+-----+---------+------+--------+-------------------+----------+-----+--------------------+-----+-----+
| symbol|counterparty_userid|side|             orderid| fee|label|last_deal|userid|fee_rate|couterparty_orderid|      size|price|               logid|funds|   id|
+-------+-------------------+----+--------------------+----+-----+---------+------+--------+-------------------+----------+-----+--------------------+-----+-----+
|HOT-ETH|             858368|NULL|5e3b6f8d6e350a000...|0.00|    0|        1|825120|    0.00|               NULL|1422832.50| 0.00|5e3b6f932e6dcc61d...| 0.00|13185|
|HOT-ETH|             858368|NULL|5e3b3e4e6e350a000...|0.00|    0|        1|355168|    0.00|               NULL|4395849.50| 0.00|5e3b3e4f2e6dcc61d...| 0.00| 7356|
|HOT-ETH|             858368|NULL|5e3b51446e350a000...|0.00|    0|        1|391744|    0.00|               NULL|7238881.50| 0.00|5e3b51482e6dcc61d...| 0.00|10368|
|HOT-ETH|             

In [52]:
df.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("s3://my-ml-buckets/workshop-prepared-data")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
data = spark.read.csv("s3://my-ml-buckets/workshop-prepared-data", header=True)  
data.drop("symbol","trade_time","account_quote_biz_no","created_at","liquidity","type","domain_id","trade_id","updated_at",\
          "counter_order_id","counter_user_id","account_base_order_id","account_base_biz_no","negative_fee_order_id",\
          "fee_currency","id","order_type","trade_source","trade_updated_at","stop","negative_fee_biz_no","account_quote_order_id",\
          "trade_type","market_price","trade_created_at","order_id","status"\
         ).printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fee: string (nullable = true)
 |-- fee_hold: string (nullable = true)
 |-- price: string (nullable = true)
 |-- funds: string (nullable = true)
 |-- side: string (nullable = true)
 |-- label: string (nullable = true)
 |-- done: string (nullable = true)
 |-- fee_rate: string (nullable = true)
 |-- force_taker: string (nullable = true)
 |-- size: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- status: string (nullable = true)

In [42]:
# 转化side列
from pyspark.sql.functions import col, create_map, lit
from itertools import chain

mapping = {'buy': 2, 'sell': 1}

mapping_expr = create_map([lit(x) for x in chain(*mapping.items())])

mapping_expr.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'Column' object is not callable
Traceback (most recent call last):
TypeError: 'Column' object is not callable



In [18]:
# 切分数据集
train_df, test_df = data.randomSplit([0.9, 0.1], 42)
train_df, val_df = train_df.randomSplit([0.8, 0.2], 42)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# shuffle data
from pyspark.sql.functions import rand

def shuffle_df(df):
    df_with_rand = df.withColumn('rand', rand(seed=42)).orderBy('rand')
    df_shuffle = df_with_rand.drop('rand')
    return df_shuffle

train_df_shuffle = shuffle_df(train_df)
val_df_shuffle = shuffle_df(val_df)
test_df_shuffle = shuffle_df(test_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# 上传至s3
train_df_shuffle.coalesce(1).write.csv("s3://sagemaker-us-east-1-851108988172/detect-workshop/data_processed/xgboost_train")
val_df_shuffle.coalesce(1).write.csv("s3://sagemaker-us-east-1-851108988172/detect-workshop/data_processed/xgboost_val")
test_df_shuffle.coalesce(1).write.csv("s3://sagemaker-us-east-1-851108988172/detect-workshop/data_processed/xgboost_test")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…